# FastAI Tabular Data Neural Network

Failed attempt to get fastai.tabular_learner() to solve this linear regression problem.

Primary problem is that tabular_learner() wants to provide categorgaical outpout, rather than a single continuious price feature

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import fastai
from fastai.basic_data import DatasetType
from fastai.tabular.data import TabularDataBunch, TabularList
from fastai.tabular.transform import FillMissing, Categorify, Normalize
from fastai.tabular.learner import tabular_learner
from fastai.metrics import accuracy, r2_score, root_mean_squared_error
import numpy as np
import pandas as pd
import random
import pydash
import torch

# from fastai.structured import *
# np.set_printoptions(threshold=50, edgeitems=20)

In [3]:
fastai.torch_core.defaults.device = 'cpu'

In [4]:
df = pd.read_csv('data/train.csv')
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [5]:
test_df = pd.read_csv('data/test.csv').fillna(0)
test_df['SalePrice'] = 0
test_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1461,20,RH,80.0,11622,Pave,0,Reg,Lvl,AllPub,...,0,0,MnPrv,0,0,6,2010,WD,Normal,0
1,1462,20,RL,81.0,14267,Pave,0,IR1,Lvl,AllPub,...,0,0,0,Gar2,12500,6,2010,WD,Normal,0
2,1463,60,RL,74.0,13830,Pave,0,IR1,Lvl,AllPub,...,0,0,MnPrv,0,0,3,2010,WD,Normal,0
3,1464,60,RL,78.0,9978,Pave,0,IR1,Lvl,AllPub,...,0,0,0,0,0,6,2010,WD,Normal,0
4,1465,120,RL,43.0,5005,Pave,0,IR1,HLS,AllPub,...,0,0,0,0,0,1,2010,WD,Normal,0


In [6]:
df.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [7]:
procs = [FillMissing, Categorify, Normalize]
cat_names = [
    'CentralAir',
    'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'HeatingQC',
    'KitchenQual', 'FireplaceQu', 'GarageQual', 'GarageCond', 'PoolQC',
    "MoSold",
    "LandSlope", "MSSubClass", "MSZoning", "Street", "Alley", "LotShape", "LandContour", "Utilities",
    "LotConfig", "Neighborhood", "BldgType", "HouseStyle", "RoofStyle", "RoofMatl", "MasVnrType", "Foundation",
    "BsmtExposure", "Heating", "Functional", "GarageFinish", "Fence", "SaleType", "SaleCondition",
    "MiscFeature", "Electrical", "GarageType", "PavedDrive",
    "Condition1", "Condition2",
    "Exterior1st", "Exterior2nd",
    "BsmtFinType1", "BsmtFinType2",
]
cont_names = [ x for x in df.columns if x not in cat_names and x not in ['Id'] ]

In [8]:
valid_idx  = random.sample( range(0, len(df)), len(df)//5)

In [9]:
# max_log_y = np.log( np.max(df['SalePrice'])*1.2 )
max_y   = np.max(df['SalePrice'])*1.2
y_range = torch.tensor([0, max_y])

In [19]:
data = TabularDataBunch.from_df(
    './data/', df, 
    dep_var='SalePrice', 
    valid_idx=valid_idx,
    test_df=test_df,
    procs=[FillMissing, Categorify, Normalize], 
    cat_names=cat_names,
    cont_names=cont_names
)
# data.add_test(TabularList.from_df(
#     test_df,
#     procs=[FillMissing, Categorify, Normalize],     
#     cat_names=cat_names,
#     cont_names=cont_names,
# ))
data

TabularDataBunch;

Train: LabelList (1168 items)
x: TabularList
CentralAir Y; ExterQual Gd; ExterCond TA; BsmtQual Gd; BsmtCond TA; HeatingQC Ex; KitchenQual Gd; FireplaceQu #na#; GarageQual TA; GarageCond TA; PoolQC #na#; MoSold 2; LandSlope Gtl; MSSubClass 60; MSZoning RL; Street Pave; Alley #na#; LotShape Reg; LandContour Lvl; Utilities AllPub; LotConfig Inside; Neighborhood CollgCr; BldgType 1Fam; HouseStyle 2Story; RoofStyle Gable; RoofMatl CompShg; MasVnrType BrkFace; Foundation PConc; BsmtExposure No; Heating GasA; Functional Typ; GarageFinish RFn; Fence #na#; SaleType WD; SaleCondition Normal; MiscFeature #na#; Electrical SBrkr; GarageType Attchd; PavedDrive Y; Condition1 Norm; Condition2 Norm; Exterior1st VinylSd; Exterior2nd VinylSd; BsmtFinType1 GLQ; BsmtFinType2 Unf; LotFrontage_na False; MasVnrArea_na False; GarageYrBlt_na False; LotFrontage -0.2159; LotArea -0.2337; OverallQual 0.6479; OverallCond -0.5104; YearBuilt 1.0415; YearRemodAdd 0.8728; MasVnrArea 0.5256; BsmtFinS

In [25]:
learn = tabular_learner(data=data, layers=[200,200,100,100,200], 
#  metrics=[root_mean_squared_error], # AssertionError: Expected output and target to have the same number of elements but got 36608 and 64.
#  use_bn=False,
   y_range=y_range
)
learn

Learner(data=TabularDataBunch;

Train: LabelList (1168 items)
x: TabularList
CentralAir Y; ExterQual Gd; ExterCond TA; BsmtQual Gd; BsmtCond TA; HeatingQC Ex; KitchenQual Gd; FireplaceQu #na#; GarageQual TA; GarageCond TA; PoolQC #na#; MoSold 2; LandSlope Gtl; MSSubClass 60; MSZoning RL; Street Pave; Alley #na#; LotShape Reg; LandContour Lvl; Utilities AllPub; LotConfig Inside; Neighborhood CollgCr; BldgType 1Fam; HouseStyle 2Story; RoofStyle Gable; RoofMatl CompShg; MasVnrType BrkFace; Foundation PConc; BsmtExposure No; Heating GasA; Functional Typ; GarageFinish RFn; Fence #na#; SaleType WD; SaleCondition Normal; MiscFeature #na#; Electrical SBrkr; GarageType Attchd; PavedDrive Y; Condition1 Norm; Condition2 Norm; Exterior1st VinylSd; Exterior2nd VinylSd; BsmtFinType1 GLQ; BsmtFinType2 Unf; LotFrontage_na False; MasVnrArea_na False; GarageYrBlt_na False; LotFrontage -0.2159; LotArea -0.2337; OverallQual 0.6479; OverallCond -0.5104; YearBuilt 1.0415; YearRemodAdd 0.8728; MasVnrArea 0.5

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [26]:
learn.fit_one_cycle(20, 5e-03)

epoch,train_loss,valid_loss,time
0,304302.500000,120169.531250,00:01
1,249140.875000,181492.156250,00:01
2,194650.093750,213500.265625,00:01
3,154031.296875,233984.343750,00:01
4,131346.421875,226348.687500,00:01
5,112556.578125,218686.468750,00:01
6,91601.460938,204381.171875,00:01
7,71824.023438,200533.562500,00:01
8,53730.289062,190650.687500,00:01
9,39121.613281,192027.718750,00:01


In [35]:
# run prediction on validation set
valid_predicts, _ = learn.get_preds(ds_type=DatasetType.Valid)
valid_probs       = np.array(valid_predicts[:,1])
# valid_targets     = df.loc[valid_idx].target.values
# valid_score       = roc_auc_score(valid_targets, valid_probs)

In [28]:
valid_predicts, _ = learn.get_preds(ds_type=DatasetType.Test)

In [31]:
valid_predicts.shape  # wrong size! should be (1459, 1)

torch.Size([1459, 572])